# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome(executable_path='./chromedriver',options=chrome_options)
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print('網頁更新中...')
    time.sleep(2)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
print('到達頁面底端')    

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 520後去留　財政部長蘇建榮尊重安排

▲財政部新春記者會。（圖／記者吳靜君攝）
記者吳靜君／台北報導
財政部今（20）日舉行新春記者會，由於總統大選才剛剛落幕，行政院已經總辭獲得留任，但是520之後，內閣還要再總辭一次，對於去留的看法。財政部長蘇建榮僅表示，520之後會尊重總統、行政院長的安排，只要在這個位置上就會做應該做的事情。
外界也關心，「蘇部長」在任內想要留下些什麼？整個國家的財政部在收入面扮演重要角色，除了稅制上的改革之外，稅政上的改革也是非常重要。
蘇建榮指出，過去他所提過的改革重點包含結構調整、稅基擴大、以及數位稅的問題，這些都不是短期內可以完成。就稅基的部分來說，在數位經濟之下，財政部積極利用大數據、以及掌握地下經濟等，對於稅負公平是一種貢獻。
他認為，稅負的合理是垂直公平與水平公平，如果都無法做到水平公平，那麼談垂直公平就沒有意義，所以稅基擴大是相當重要的，目前財政部也正積極推動。
結構調整方面，目前所得稅占50%、消費稅占35%，財產稅只有5%多，但是相較於國際，歐盟、日、美稅制結構消費與所得是貢獻差不多的，這不只是稅法的改變，稅基擴大也是一種方式，不只是改變法制，從稽徵行政、數位上也可以達到這個目的。
但是稅基擴大不是一下子就能發揮，稅基的部分要有效掌握，包含地下經濟、跨境電商的營業稅申報，其他在網路上銷售行為，稽徵上可以有效查核。
蘇建榮自2018年7月上任，最自豪的事情是完成兩件事情，其一是實質減債；另一方面就是保障公股事業員工的權益包含公股銀與台酒等都簽團體協約，且泛公股銀除華銀外，也皆有員工持股信託。
----------------------------------------------------------------------

[1] 春安勞檢北區客運站　驚見客運司機單月加班90小時

記者余弦妙／台北報導
勞動部從1月3日起執行「春安勞檢」，今（20）日舉辦全台北、中、南歲末春安聯合檢查，北區由勞動部政務次長劉士豪帶隊勞檢國道客運業，其中共有3家違反工時過長規定，甚至還有司機單月加班達90小時，遠超過勞基法規定的單月46小時上限。
勞動部指出，今天針對新北市板橋區客運

▲LG的新式折疊式手機曝光。（圖／取自Letsgodigital）
記者謝仁傑／綜合報導
繼三星Galaxy Fold、華為Mate X等新款智慧型手機在2019年引發熱度後，據外媒傳出LG有關折疊式手機的多項技術專利也已通過，其中，手機螢幕可環繞延伸到背面特徵，最受矚目。
近幾年，「折疊式手機」成為手機市場主流，各大手機廠商紛紛開始投入創新型折疊螢幕手機的研發。
▲LG專利草圖。（圖／取自Letsgodigital）
依據專利設計草圖可以看出，在手機底部與左側採用細邊框設計，頂部邊框相對較厚，並內建閃光燈的雙鏡頭。機身右側無邊框，手機螢幕可一直環繞延伸到背面，類似小米MIX Alpha的環繞螢幕設計。從專利草圖中，看起來搭載兩個前鏡頭，但並沒有後置鏡頭。背部螢幕可自由翻轉，擴增介面。
----------------------------------------------------------------------

[12] 台股資金行情強強滾　國泰金調查：4成民眾看好上半年衝12500點

▲國泰金對民眾進行上半年台股高點與低點的預期調查顯示，4成民眾預期上半年台股高點會落在12,000點至12,500點區間。（示意圖／記者呂佳賢攝）
記者紀佳妘／台北報導
國泰金（2882）今（20）日發布最新調查，民眾對於股市樂觀情緒普遍走升，顯示民眾對風險偏好的程度增加，且有40.2%的民眾預期2020上半年台股高點會落在12,000點至12,500點區間。
根據證交所數據顯示，台灣加權股價指數2019年封關收在11,997點，全年指數漲幅達23%，創近10年最大年度漲幅，年中指數曾一度衝高至12,126點，創下29年波段新高。
國泰金發布2020年1月份國民經濟信心調查結果，民眾對股市的樂觀情緒走升，對風險偏好的程度也增加，主要是中美達成第一階段貿易協議，且半導體營收成長強勁，投資人風險情緒上揚，台股站穩12,000點關卡。
調查也指出，民眾股市樂觀指數大幅走高至-1.5，風險偏好指數也上揚至5.1，顯示中美達成初步協議與景氣穩健走強，使得民眾對股市樂觀情緒與風險偏好提升。
調查結果指出，40.2%的民眾預期2020上半年台股高點會落在12,000點至12,500點區間，18.5%的民眾則認為落在12,500點至13,000點，另有8.3%的民眾認為會落在13,

▲法國精品龍頭LVMH執行長阿爾諾（Bernard Arnault）。（圖／路透社）
實習記者崔子柔／綜合報導
全球首富再次換人做！由於電商龍頭亞馬遜週五（17日）股價下跌0.7%，導致亞馬遜創辦人貝佐斯（Jeff Bezos）的身價下跌，而精品龍頭LVMH集團股價上漲0.7%，使得法國LVMH集團執行長阿爾諾（Bernard Arnault）身家超越貝佐斯，成為新的全球首富。
據美國《富比士》（Forbes）雜誌報導，阿爾諾因LVMH股價上漲，資產增加19億美元，貝佐斯則因亞馬遜股價下跌損失7.6億美元（約228億元台幣），目前阿爾諾資產達1170億美元，已經超越貝佐斯的1156億美元（約3兆4670億元台幣）。
阿爾諾去年6月成為《富比士》全球富豪榜中第3位，之後多次越過貝佐斯成為全球首富，因2人資產差距不超過20億美元，他們在未來應會持續爭奪首富寶座。
微軟創辦人比爾蓋茲（Bill Gates）目前是全球第三大富豪，擁1100億美元資產，曾在去年11月回歸首富之座，當時他自嘲「錢捐得不夠快」，邀請各大富豪一同捐款，若他先前沒有將358億美元捐給慈善機構，如今仍是全球首富。
▼蓋茲每年捐款超過50億美元。（圖／路透）
----------------------------------------------------------------------

[25] 景氣很不同！兆豐董座看景氣　台商回台投資「有感的經濟成長」

▲兆豐金董事長張兆順。（圖／記者吳靜君攝）
記者吳靜君／台北報導
兆豐銀舉辦尾牙，兆豐金暨兆豐銀董事長張兆順今（19）日接受媒體採訪時表示，台商回台投資帶動的經濟成長，與過去出口帶動經濟成長很不同，民眾會有感。
張兆順接受媒體採訪時，針對景氣的看法表示，今年整個世界景氣持平，因為貿易戰已經到一個階段，還未完全消除，未來可能會有科技戰、貨幣戰，若打到貨幣戰就很恐怖，就會有「制裁」，只要客戶用美元交易，對國內的銀行業就很麻煩，他舉例假設一個新加坡商開信用狀給國內的企業，國內企業到國內銀行押匯，清算機制會落到兆豐銀，所以各銀行都要確保並且能偵測到美元是否會流到制裁國家。
所以世界銀行對於全球景氣稍微調降，現在中美貿易已經簽了第一階段的協議，就有可能會比較緩和，所以今年的景氣應該與去年差不多。
但是台灣就很不同，張兆順說，台商過去的資金都在海外

▲台灣中油公司今（18）日舉行僱用人員甄試，全台有1298人進入第二試關卡。（圖／台灣中油提供）
記者林淑慧／台北報導
企業尾牙一波波，年後轉職潮蠢動。台灣中油公司今（18）日舉行僱用人員甄試，全台有1298人進入第二試關卡，本次甄試預計錄取672人，並將在2月14日公告錄取名單，決定誰能順利取得「鐵飯碗」。
台灣中油公司108年僱用人員甄試第二試，今(18日)、明兩日在台北、高雄2個考區舉行，報名人數13,972人，進入第二試人數1,298人，預計錄取672人；同日舉辦108年離島及偏遠地區僱用人員甄試第二試，報名人數699人，進入第二試人數121人，預計錄取44人。
台灣中油表示，第二試為口試及現場測試，現場測試項目包含現場適應性模擬及體能測驗，依類別手提滅火器分別跑走50公尺或70公尺、徒手跑走800公尺、負重30公斤重物跑走50公尺、辨色實作模擬及辨識標示牌等項目。
另外，為了使現場測試項目符合現場工作體能需要，本次「現場適應性模擬及體能測驗」項目除維持上下鷹架，並增加應考人須上拉18公斤重物至鷹架平台之測試。第二試(口試)占總成績30%，現場測試則採合格制，不另予計分，凡任一項未通過即遭淘汰。
台灣中油表示，本次甄試預計於109年2月14日公告錄取名單，經第二試(現場測試)合格者，依各該類別同一工作地區或睦鄰區域(含分區)之錄取名額，按甄試總成績高低順序錄取，並依志願分發至服務單位訓練，訓練6個月期滿成績合格後正式僱用。
----------------------------------------------------------------------

[35] 雞排祭品文爆量兌現　帶動夜市券回收量突破500萬張大關

▲經濟部表示夜市券促進國內消費有成，夜市攤商們的營業額都提高2至3成，合計帶動創造超過 7.7 億元的內需消費效益。(圖／記者徐文彬攝)
記者林淑慧／台北報導
2020總統大選落幕，各地雞排祭品文爆量兌現，帶動夜市券回收量。依據經濟部最新統計，全台夜市券目前已經回收突破500萬張，預估可創造超過7.7億元的內需消費效益。
廣受自由行旅客及夜市攤商歡迎的夜市券政策，在行動支付業者大方祭出回饋好禮後，加上2020總統大選的雞排祭品文兌現量，帶動消費金額持續成長。
經濟部官員指出，全台夜市券回收冠軍，依舊由花蓮的東大門夜市蟬聯，目前已回

▲ 新台幣匯率走勢與股市息息相關。（圖／路透）
記者李瑞瑾／台北報導
總統大選順利落幕，金融市場股匯齊揚、新台幣兌美元匯率也終於突破30元大關，來到29字頭。展望匯市，專家認為選後政治局勢底定、不確定性因素漸消散，台幣第一季可望持續強勢。
但匯銀人士也指出，新台幣雖然維持相對升值姿態，但幅度不會太多，會處於「溫和的相對高點」。短線觀察新台幣匯率走勢可以關注兩個時間點，第一是中美第一階段協議簽訂，如果結果不如預期，可能讓新台幣有回檔機會。
而在農曆年後資金流回銀行體系，升值就會稍微趨緩；此外，年後紅包行情也告一段落，若股市出現獲利了結，也可能讓新台幣向下修正。
另一方面，今年第一季美元仍有逐步回調的可能性。川普希望聯準會維持相對寬鬆的貨幣政策，甚至出來買短債，因此市場資金相對比較浮濫，錢從美國流出，而亞洲貨幣反映節慶季節性因素，則會相對美元升值。
▲ 亞幣因季節性因素，在第一季將表現比美元強勢。圖為泰國潑水節。（資料照／泰國觀光局）
以整體亞洲資金需求來看，除了華人過農曆年，到了4月則是東南亞國家過浴佛節、潑水節等，亞洲第一季都可以吸引國際資金停泊，有資金需求也會讓幣值相對比較走升。
但到了第二季就要留意資金動向改變。匯銀人士提醒，第二季國際資金目光將轉回到美國身上，主要因美國步入選舉，選舉前半年需要資金到位，競選連任的川普也會把打擊重點從國際上移回國內，並將著重在國內經濟發展。
----------------------------------------------------------------------

[45] 小英勝選財經篇／工商團體呼籲打造友善投資環境、勞團要求「兌現基本工資3萬承諾」

▲工商協進會理事長林伯豐。（圖／記者陳政錄攝）
記者余弦妙／台北報導
2020總統大選由民進黨候選人蔡英文高票當選，這也意味著她將背負更大的期望，工商團體呼籲放寬勞動政策、打造友善投資環境、改善兩岸經貿關係；勞團則是提醒蔡英文要履行基本工資3萬元的承諾目標，不要因為連任了，就無後顧之憂地跳票。
另外，林伯豐說，希望蔡政府要重新檢視能源政策，全球都在減媒，只有台灣增加，建議核電議題應該要再重新檢視，台灣如果不使用核電，這對台灣的經濟發展來說不是一個好現象。
林伯豐也希望蔡政府應該要好好與中國大陸對談，且與國際簽署CPTPP、RCEP，不要只是喊口號。
林伯豐更提

▲日月光董事長張虔生。（圖／日月光提供）
記者周康玉／台北報導
日月光高雄廠今（17）日舉行尾牙，董事長張虔生趁今日加發給12000名基層員工1萬元獎金，總金額超過1.2億元，等於是在春節前提前領一個大紅包。
日月光集團高雄廠自2015年至今，連六年加發產線基層人員一萬元獎金，更連續三年調薪，以實質所得回饋同仁，並鼓勵持續精進專業、學習成長，強化職場競爭力，不僅提振士氣、帶動公司的向心力與凝聚力，更成為公司連年成長的動力。
展望2020年，日月光將持續厚植高端先進封裝技術，發展自動化科技與智慧製造，強化半導體產業競爭力，開創更多就業機會，讓優秀的產業專才留在台灣。
----------------------------------------------------------------------

[53] 【富豪排行大搬風】79歲蟬聯台灣女富豪　丈夫生前也數不清自己擁有多少土地

根據《富比士》所公布的2020年「台灣50富豪榜」，今年79歲的林張素娥以新台幣599億元的身家名列第18，與前次相比，資產增加65.82億元、排名進步4名，不只蟬聯台灣女首富，也是榜內唯二的女性。
林張素娥是聯邦企業集團創辦人林榮三的遺孀，在2015年繼承丈夫所留下的龐大遺產。林榮三生前就是《富比士》台灣富豪排行榜上的常客，在世時橫跨政商兩界，曾任監察院副院長、監委、增額立委，也是聯邦集團、《自由時報》創辦人，由於和兄長──台灣地王林堉璘同樣靠房地產發跡，據說生前連自己都數不清擁有多少土地，雄厚的財力由此可見。
「台灣50富豪榜」的第一代台灣女首富是廣達董事長林百里的元配何莎，在廣達成立初期，她曾帶著兒子跑號子賣股籌錢，後來也陪林百里跑遍樹林泰山一帶爭取訂單，堪稱有功在身，她手上握有的16萬張廣達股票也讓她一度成為女首富，不過，由於近年陸續將股票贈與2個兒子，如今已跌出排行榜。
第二代女首富則是「經營之神」王永慶的二房次女王雪紅，她所創立的宏達電在2005年成為台灣上市公司股王，到了2011年，《富比士》公布世界億萬富豪排行榜，王雪紅與丈夫陳文琦還一度以新台幣2,003億元的身價，將鴻海董事長郭台銘擠下台灣首富的寶座。
不過，之後宏達電就開始走下坡，2018年更被迫將旗下手機部門賣給Google，且宣稱發展AR及VR等新事業也未見曙光，事業一度消沉，直到最近威盛電子旗下IC設計

▲台指期1月17日走勢。（圖／取自期交所官網）
記者余弦妙／台北報導
台指期今（17）日開盤上漲51點，來到12,103點，雖然盤中一度跌至12,029點，不過隨後拉升，終場以12,072點作收，上漲20點，與現貨相較，逆價差18.29點。展望後市，統一期貨分析師劉恆誌分析，下周一封關，不太有大漲大跌盤勢出現，除非本周末出現大事件，不然就是要等年後開紅盤，再操作。
劉恆誌表示，雖然昨天美股上漲，但並沒有反映到今天的台股，且震盪的幅度並不大，以指數的高點來看來到12,118點，而低點則是來到12,029點，震盪幅度在89點，並未超過百點，主要是越靠近封關，投資人就不太會買進新股，且大部分的人甚至會把資金從股市抽出，所以農曆年前不太有機會出現明顯的大漲大跌，除非在周末國際間出現大事件，例如之前美伊戰爭這類消息才會再度出現恐慌型的拋售。
劉恆誌也說，目前中美第一階段協議已經順利結束，接著就是等第二階段的談判時程，因此目前從國際情勢上來看也不太有不利消息。
劉恆誌說，接下來大家會關注的就是美股的第四季財報，現在銀行股已經開跑了，但銀行股與台灣供應鏈的相關性比較不大，頂多只能先看銀行股的表示如何，再來是29日、30日的蘋果、FaceBook、波音等重磅財報公布。
----------------------------------------------------------------------

[62] 活水回來了！台商資金匯回申請破5百億元　會計師：投資計畫1月底前要備查

▲台商依據境外資金匯回專法申請匯回資金，已經破500億元。（圖／pixabay、示意圖）
記者吳靜君／台北報導
境外資金會回管運用與課稅條例去（2019）年上路至今已經5個月，根據財政部統計，自上路至1月14日（週二）止，申請案件已經達199件，申請金額達517億元。KPMG安侯建業聯合會計師事務所表示，若要匯回投資，投資計劃要在1月底前備查，以免被補稅。
假使未依規定辦理備查或未於通知期限內補報者，經濟部將通知該申請人戶籍所在地或登記地稽徵機關就未報備查的資金，必須按20%稅率補徵差額稅款。
但是應該檢附的文件，會計師提醒有二類：
一，實質投資者應檢附投資計畫，應該檢附投資計畫備查申請書、投資計畫核准函、被投資事業最近一次設立登記；投資計劃期間持股未異動證明等文件。
二，間接實質投資，應檢附

▲社區型別墅建案「美麗山林」擁有4萬2千坪樹海環繞，深受不少樂活購屋族喜愛。（採訪撰稿／記者黃稜涵；攝影剪輯／記者謝婷婷）
記者浦靖祤／台北報導
擁有超大空間還有前庭後院，住在氣派的別墅裡，一直是許多民眾的夢想，不過怎麼樣才能安心享受別墅生活，現在可以有更聰明的選擇。房地產專家田大權表示，「過去看到的獨棟別墅容易有安全上的問題，因為社區管理技術和knowhow並不好，所以造成很多公共設施閒置或荒廢，現在大家比較喜歡社區別墅，因為它的管理與各方面的社區機能，都相對比較成熟比較好。」
挑選別墅除了以社區型為首要考量，別墅的地點更是重要，離塵不離城這剛剛好的距離，讓生活品質大大加分。田大權指出，「未來看好的地區會是在汐止這一塊，因為大南港生活圈整個擴張出去，對於汐止的帶動效應，這幾年的市場表現就看得到，最重要的是東區門戶計畫，有高達數千億各種產業發展的聚落。」
近期在大南港地區最受矚目的社區別墅案「美麗山林」，擁有4萬2千坪樹海環繞，全區採低密度開發，在這裡不僅享有悠閒生活，基地位置離市區也相當的近，不失生活機能。 
效法加拿大園藝美學，建案規劃萬坪公園，就好比布查花園般引人入勝，讓別墅住家與大自然融合，創造在城市中難能可貴的悠閒自在。甲山林協理蘇同福表示，「美麗山林獨棟的外觀設計，擁有11米大面寬以及私人雙車位，大面寬的設計引景映景，把光跟風融入室內，別墅在自然的環境裡，住戶可以享受自在呼吸的空氣。我們都說住進美麗山林的客人，可以年輕10歲，也是這個產品非常具有的特色。」
▲已購客戶劉先生與太太選擇社區型別墅享受悠閒退休生活。▼「美麗山林」規劃千坪頂級休閒會館。（圖／記者姜國輝攝）
已購客戶劉先生分享，「到了這個年紀想說想為了退休生活規劃，心目中最理想的就是透天別墅，但林口、桃園地區較偏遠，台北的陽明山我們也不是很習慣，覺得生活機能不好，剛好看到美麗山林這個建案，第一次來看就付訂金了， 一個禮拜就簽約。這裡真的非常棒，因為會所的規劃和氣氛，都營造的跟一般平地的產品不一樣，可以說是物超所值的產品，以後也買不到了！」
社區住戶專屬的千坪休閒會館，除了頂級的宴會廳，也貼心打造景觀健身房、瑜伽教室還有娛樂遊憩空間，甚至還有媲美電影院的視聽室以及水池Lounge Bar，讓許多購屋客都難以想像，不用遠離台北市，就能貼近夢想中的別墅生活。
▲景觀優美的水池Lounge Bar。▼「

▲蘋果目前仍展現固守用戶隱私的決心。（圖／路透社）
記者謝仁傑／綜合報導
據外媒最新報導指出，從1月20日開始，蘇格蘭警方可為了破案，開始使用所謂的「網路資訊亭」（cyber kiosks），檢查和提取智慧型手機中的相關資料。
調查人員可以透過這些「資訊亭」破解受害者、嫌疑犯和證人的設備，無論這些設備是在調查過程中自願上交還是沒收的，蘇格蘭法律已經給予執法部門拿走涉案人員的設備，並且無限期扣押的權力。
與蘇格蘭情況相似，美國執法部門也開始嘗試從智慧手機當中提取資料。FBI和檢察總長William Barr，要求蘋果公司協助解鎖恐怖分子阿爾薩姆拉尼（Mohammed Saeed Alshamrami）的兩部iPhone。
然而，蘋果表示，目前並不會為政府開設特權，即不會在系統中留下後門，讓用戶的資料可以隨意暴露。
----------------------------------------------------------------------

[82] 永慶房產集團嘉南區聯合尾牙　慶祝聯賣推升業績成長5成！

▲台南市長黃偉哲(中)，與永慶房產集團加盟事業處董事長劉炳耀(右七)一同祝賀嘉南區永慶加盟三品牌創下業績成長5成的佳績。
圖、文／永慶房產集團提供
永慶房產集團2019年績效大爆發，10場尾牙宴一路慶祝，昨(1/16)慶功熱力延燒到台南永康，席開近百桌，宴請嘉南區永慶不動產、有巢氏房屋、台慶不動產的員工與眷屬，慶祝年業績成長5成！台南市長黃偉哲更親臨現場，與永慶房產集團加盟事業處董事長劉炳耀一同祝賀菁英們的好成績！
尾牙一開場就由知名「御善喬國際舞團」帶來的熱情舞蹈，瞬間引爆熱鬧氣氛；華麗舞姿，搭配超嗨的音樂與燈光秀，觸動全場觀眾聽覺與視覺，尖叫聲不斷！
▲嘉南區尾牙現場邀請到啦啦隊表演，為嘉南區2019年亮眼的成績喝采！
168份禮 Gogoro、東京來回機票、iPhone11 讓員工一路發！
嘉南區經管會更準備近百萬元的好禮，除了環保的gogoro、夢幻3C新品的iPhone11、最實用的Dyson空氣清淨機、電視、任天堂Switch，還有東京來回機票等多項好禮！永慶不動產嘉南區經管會會長呂啟綸分享：「特別準備了『168』份獎品，希望幸運數字讓所有同仁業績『一路發』！」
為了增加趣味性與獎項，現場還安排限時喝果汁、走天堂路等趣味競賽，優勝者可獲得

▲iPhone 12渲染圖。（圖／Phonearena）
記者謝仁傑／綜合報導
分析師補充說，iPhone 12 Pro的後置鏡頭系統將採用ToF飛時測距，使拍攝時具3D感測功能。他們還預計iPhone 12 Pro將配備6GB的隨機存取記憶體，高於iPhone 11 Pro的4GB。
同時分析師們也預測，最大的變化可能在明年出現。他們表示蘋果有可能在2021年從至少一款iPhone機型上移除Lightning插孔，呼應了知名蘋果產品分析師郭明錤上個月的預測。
----------------------------------------------------------------------

[93] 數位浪潮來襲！國銀近5年新設分行趕不上裁撤家數　2銀行放棄不開業

▲根據金管會統計，過去5年各家銀行裁撤分行總家數為80家、裁撤總家數則為36家。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
金管會最新統計指出，2015年至2019年國銀裁撤總分行數為80家，同時期核准新設總分行數則為36家，顯示各銀行新設分行趕不上裁撤分行的速度；此外，凱基銀、京城銀分別在2017年向金管會遞件申請新設3家分行，不過皆在2019年放棄不開業。
根據金管會銀行局統計，截至2019年11月30日，本國銀行分支機構家數共有3,404家，國內每10萬成年人平均擁有本國銀行分行家數為17家。
觀察近5年來國銀分支機構表現，2015年核准新設4家分行、裁撤23家分行，2016年增設4家、裁撤22家分行，2017年增設9家分行、裁撤14家分行，2018年增設10家分行、裁撤15家分行，2019年增設9家分行、裁撤6家分行。
從數據來看，過去5年各家銀行裁撤分行總家數為80家、裁撤總家數則為36家，由此可見，銀行裁撤總分行數比金管會核准新設分行足足多了2.2倍，換句話說，各銀行新設分行趕不上裁撤分行的速度。
金管會銀行局表示，目前金管會已核准凱基銀行、陽信銀行、京城銀行可設立4家分行，其中凱基銀行在2017年申請新設2家分行，不過由於找不到適合的地點、因應數位化等問題，自行評估效益，後在2019年向金管會表達決定不開業。
至於京城銀行也是在2017年核准新設分行，不過到了2019年也遲遲尚未有動作，銀行局指出，按照規定，金管會核准新設分行後，1年內必須要開業，超過1年的話就要申請延長，若沒